<a href="https://colab.research.google.com/github/21-10-4/zxing/blob/master/pytorch%E5%85%A5%E9%97%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../dataset/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ../dataset/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../dataset/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ../dataset/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../dataset/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ../dataset/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../dataset/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../dataset/mnist/MNIST/raw

[1,   300] loss: 0.744
[1,   600] loss: 0.233
[1,   900] loss: 0.168
accuracy on test set: 95 % 
[2,   300] loss: 0.126
[2,   600] loss: 0.110
[2,   900] loss: 0.099
accuracy on test set: 97 % 
[3,   300] loss: 0.082
[3,   600] loss: 0.089


KeyboardInterrupt: ignored

In [1]:
!nvidia-smi

Fri Jan  6 15:52:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

# prepare dataset

batch_size = 64
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])  # 归一化,均值和方差

train_dataset = datasets.MNIST(root='../dataset/mnist/', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataset = datasets.MNIST(root='../dataset/mnist/', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)


# design model using class


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1,10,kernel_size=5)
        self.conv2 = torch.nn.Conv2d(10,20,kernel_size=5)
        self.pooling = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Linear(320,20) # 28*28经过5*5卷积核，变为24*24,讲过pooling，变为12*12；再8*8,4*4
        self.fc2 = torch.nn.Linear(20,10)
        # 20*4*4 = 320

    def forward(self, x):
        batch_size = x.size(0) # 样本个数
        x = F.relu(self.pooling(self.conv1(x)))
        x = F.relu(self.pooling(self.conv2(x)))
        x = x.view(batch_size,-1) # flatten
        x = self.fc(x)
        return self.fc2(x) # 最后一层不做激活，不进行非线性变换


model = Net()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# construct loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


# training cycle forward, backward, update


def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        # 获得一个批次的数据和标签
        inputs, target = data
        inputs, target = inputs.to(device),target.to(device)
        # 获得模型预测结果(64, 10)
        outputs = model(inputs)
        # 交叉熵代价函数outputs(64,10),target（64）
        loss = criterion(outputs, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 300))
            running_loss = 0.0


def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device),labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)  # dim = 1 列是第0个维度，行是第1个维度
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  # 张量之间的比较运算
    print('accuracy on test set: %d %% ' % (100 * correct / total))


if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../dataset/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ../dataset/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../dataset/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ../dataset/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../dataset/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ../dataset/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../dataset/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../dataset/mnist/MNIST/raw

[1,   300] loss: 0.750
[1,   600] loss: 0.217
[1,   900] loss: 0.145
accuracy on test set: 95 % 
[2,   300] loss: 0.114
[2,   600] loss: 0.107
[2,   900] loss: 0.092
accuracy on test set: 97 % 
[3,   300] loss: 0.080
[3,   600] loss: 0.075
[3,   900] loss: 0.079
accuracy on test set: 97 % 
[4,   300] loss: 0.065
[4,   600] loss: 0.065
[4,   900] loss: 0.065
accuracy on test set: 98 % 
[5,   300] loss: 0.055
[5,   600] loss: 0.060
[5,   900] loss: 0.057
accuracy on test set: 98 % 
[6,   300] loss: 0.052
[6,   600] loss: 0.046
[6,   900] loss: 0.055
accuracy on test set: 98 % 
[7,   300] loss: 0.049
[7,   600] loss: 0.043
[7,   900] loss: 0.047
accuracy on test set: 98 % 
[8,   300] loss: 0.043
[8,   600] loss: 0.044
[8,   900] loss: 0.043
accuracy on test set: 98 % 
[9,   300] loss: 0.039
[9,   600] loss: 0.041
[9,   900] loss: 0.038
accuracy on test set: 98 % 
[10,   300] loss: 0.032
[10,   6